In [14]:
import sys
sys.path.append('../')
from setting import config_read
from mapping import *

In [15]:
from konlpy.tag import Okt
from owlready2 import get_ontology
from elasticsearch import Elasticsearch

In [16]:
# Read config file
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = get_ontology(data_path).load()

# Elasticsearch
index_name = config['elasticsearch']['name']
server_ip = config['elasticsearch']['ip']
es = Elasticsearch(server_ip)

In [17]:
okt = Okt()

query = input("Question: ") # 용감한 형제가 작곡한 노래의 장르는?
query_pos = okt.pos(query, norm=True)

query_terms = [q[0] for q in query_pos if q[1] not in ["Josa", "Punctuation"]]

In [18]:
query_patitions = query_patitioning(query_terms)

print("Query Term Set:", query_terms)
print("Query Partitions Set:")
for qp in query_patitions:
    print(qp)

Query Term Set: ['용감한', '형제', '작곡', '노래', '장르']
Query Partitions Set:
['용감한 형제 작곡 노래 장르']
['용감한', '형제 작곡 노래 장르']
['용감한 형제', '작곡 노래 장르']
['용감한 형제 작곡', '노래 장르']
['용감한 형제 작곡 노래', '장르']
['용감한', '형제', '작곡 노래 장르']
['용감한', '형제 작곡', '노래 장르']
['용감한', '형제 작곡 노래', '장르']
['용감한 형제', '작곡', '노래 장르']
['용감한 형제', '작곡 노래', '장르']
['용감한 형제 작곡', '노래', '장르']
['용감한', '형제', '작곡', '노래 장르']
['용감한', '형제', '작곡 노래', '장르']
['용감한', '형제 작곡', '노래', '장르']
['용감한 형제', '작곡', '노래', '장르']
['용감한', '형제', '작곡', '노래', '장르']


In [19]:
p_star, score_list = partition_scoring(es, index_name, query_patitions)
print(score_list)
print(p_star)

[0, 0, 7.171753, 0, 6.5522356, 0, 0, 6.5522356, 14.343506, 13.7239886, 13.7239886, 7.171753, 6.5522356, 13.7239886, 28.0674946, 20.8957416]
['용감한 형제', '작곡', '노래', '장르']


In [20]:
# candidates_list = []
# for q in p_star:
#     earch_query = {"query":{"term":{"Annotation Values.keyword": q}}}
#     candidates = es.search(index=index_name, body=earch_query)['hits']['hits']
#     candidates_list.append(candidates)
# resource_combinations = list(itertools.product(*candidates_list))

In [21]:
# Resource Mapping and List Resource Combinations
# candidates_list = [onto.search(label=query) for query in p_star]
# resource_combinations = list(itertools.product(*candidates_list))
# resource_combinations